# Create input for DeCiFer

This notebook provides an exemplary script for generating the input for DeCiFer from multiple bulk tumour samples and matched normal sample when using:
- HATCHet for inferring allele-specific copy numbers
- Varscan for inverring somatic single-nucleotide variants (SNVs)
- BCFtools for counting sequencing reads for all SNVs across all tumour samples


## 1. Required inputs

There are three required inputs that should be provided in the related variables below.

1. **`CNAs`**: `best.seg.ucn` file with HATCHet's inferred allele-specific copy numbers.
2. **`SNVs`**: Varscan output TSV file with somatic SNVs (mandatory fields include `chrom`, `position`, `ref`, and `var`) called for every sample independently. 
3. **`MPIs`**: BCFtools files of read counts generated for every tumour sample independently. Specifically, each file `${SAMPLE}.mpileup.tsv` should be generated with a command equivalent to the follow for every sample `${SAM}`, with reference genome `${REF}`, and SNVs file `${SNV}` from 2. above:
```shell
bcftools mpileup ${SAM} -f ${REF} -T <(cut -f1-2 ${SNVs} | grep -v position) -a INFO/AD -Ou | bcftools query -f'%CHROM\t%POS\t%REF,%ALT\t%AD\n' > ${SAMPLE}.mpileup.tsv
```

In [1]:
## Please specify the required inputs in the three related variables with the suggested format

#CNAs = '/path/to/best.seg.ucn'
#SNVs = {
#    'SAMPLE1' : '/path/to/SAMPLE1.varscan.tsv',
#    'SAMPLE2' : '/path/to/SAMPLE2.varscan.tsv',
#}
#MPIs = {
#    'SAMPLE1' : '/path/to/SAMPLE1.mpileup.tsv',
#    'SAMPLE2' : '/path/to/SAMPLE2.mpileup.tsv',
#}

CNAs = '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/hatchet_selected/SJHGG010160/results/best.seg.ucn'
SNVs = {
    'SJHGG010160_A1' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_A1.snp',
    'SJHGG010160_A2' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_A2.snp',
    'SJHGG010160_D1' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_D1.snp',
}
MPIs = {
    'SJHGG010160_A1' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_A1.mpileup.tsv',
    'SJHGG010160_A2' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_A2.mpileup.tsv',
    'SJHGG010160_D1' : '/n/fs/ragr-research/users/bjarnold/MetastaticEvol/varscan/SJHGG010160/SJHGG010160_D1.mpileup.tsv',
}

## Also, please specify the name or full path of the two generated input files for DeCiFer

INPUT_SNVs = 'decifer.input.tsv'
INPUT_PURITY = 'decifer.purity.tsv'

## Finally, the following parameters are used for variant filtering

PVALUE = 1e-03 # Maximum threshold for Varscan pvalue score, choose 1 if you want to disable it.
MINREADS = 30 # Minimum total number of reads per SNV across all samples
MAXREADS = 10000 # Maximum total number of reads per SNV across all samples

## 2. Execute the script for creating DeCiFer's input

After succesfully setting up the required inputs, the following steps can be executed directly through this python notebook (or as a python script) to create DeCiFer input. When using this jupyter notebook, simply run all the cells below

In [2]:
import sys, os
import glob
import pandas as pd
from collections import defaultdict
from collections import Counter

In [3]:
## SNVs data and read counts are properly combined and formatted

snv_df = {}
for sam, f in SNVs.items():
    snv_df[sam] = pd.read_csv(f, sep='\t')
    snv_df[sam] = snv_df[sam][snv_df[sam]['somatic_p_value'] < PVALUE]
    snv_df[sam]['snv_id'] = snv_df[sam].apply(lambda line: ".".join(map(str, [line['chrom'], line['position'], line['ref'], line['var']])), axis=1)
mpi = {}
form = (lambda p : ((p[0], int(p[1])), Counter(dict(filter(lambda v : '*' not in v[0], zip(p[2].split(','), map(int, p[3].split(','))))))))
for sam, f in MPIs.items():
    mpi[sam] = defaultdict(lambda : Counter({'A' : 0, 'C' : 0, 'G' : 0, 'T' : 0}))
    with open(f, 'r') as i:
        for l in i:
            g, u = form(l.strip().split())
            mpi[sam][g].update(u)
        mpi[sam] = dict(mpi[sam])
refvar = defaultdict(lambda : (Counter({'A' : 0, 'C' : 0, 'G' : 0, 'T' : 0}), Counter({'A' : 0, 'C' : 0, 'G' : 0, 'T' : 0})))
for sam in snv_df:
    for i, r in snv_df[sam].iterrows():
        g = (str(r['chrom']), r['position'])
        refvar[g][0].update(Counter({r['ref'] : mpi[sam][g][r['ref']]}))
        refvar[g][1].update(Counter({r['var'] : mpi[sam][g][r['var']]}))
refvar = {g : refvar[g] for g in refvar if sum(refvar[g][0].values()) > 0 and sum(refvar[g][1].values()) > 0 and MINREADS <= (sum(refvar[g][0].values()) + sum(refvar[g][1].values())) <= MAXREADS}
argmax = (lambda D : max(D.keys(), key=(lambda x : D[x])))
refvar = {g : tuple(map(argmax, refvar[g])) for g in refvar}
assert all(refvar[g][0] != refvar[g][1] for g in refvar)
gid = (lambda g : '.'.join(map(str, [g[0], g[1], refvar[g][0], refvar[g][1]])))
form = (lambda s, g : {'snv_id' : gid(g), 'Sample' : s, 'chrom' : g[0], 'position' : g[1], 'tumor_reads1' : mpi[s][g][refvar[g][0]], 'tumor_reads2' : mpi[s][g][refvar[g][1]]})
default = (lambda s, g : {'snv_id' : gid(g), 'Sample' : s, 'chrom' : g[0], 'position' : g[1], 'tumor_reads1' : 1, 'tumor_reads2' : 0})
snv_df = pd.DataFrame([form(s, g) if g in mpi[s] else default(s, g) for s in mpi for g in refvar])
selected_ids = snv_df['snv_id'].unique()
print('Number of selected SNVs: {}'.format(len(selected_ids)))
sample_index = {v:i for i, v in enumerate(snv_df['Sample'].unique())}
character_index = {v:i for i, v in enumerate(selected_ids)}

/n/fs/ragr-research/users/bjarnold/miniconda3/envs/decifer/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of selected SNVs: 471


In [4]:
## Read CNAs data and generate purity input

cna_df = pd.read_csv(CNAs, sep = '\t')
cna_df['purity'] = 1.0 - cna_df['u_normal']
purities = dict({(r['SAMPLE'], r['purity']) for i, r in cna_df.iterrows()})
with open(INPUT_PURITY, 'w') as o:
    for s in purities:
        o.write("{}\t{}\n".format(sample_index[s], purities[s]))

In [5]:
## Combine SNVs and CNAs data

discarded = 0
input_data = []
for i, snv in enumerate(selected_ids):
    highcn = False
    buff = []
    char_idx = character_index[snv]
    char_label = snv
    if i % 500 == 0: print("{} {}".format(i, len(character_index)))
    for sample in snv_df['Sample'].unique():
        sample_idx = sample_index[sample]
        
        snv_line = snv_df[(snv_df['snv_id'] == snv) & (snv_df['Sample'] == sample)].iloc[0]
        try:
            chrom = int(snv_line['chrom'])
        except ValueError:
            continue
        pos = int(snv_line['position'])
        ref = snv_line['tumor_reads1']
        var = snv_line['tumor_reads2']         
        intervals = cna_df[(cna_df['#CHR'] == chrom) & (cna_df['START'] <= pos) & (cna_df['END'] > pos) & (cna_df['SAMPLE'] == sample)]
        if len(intervals) == 0: 
            discarded += 1
            continue
        
        try:
            cn_dict = {}
            for idx in ['normal', 'clone1', 'clone2', 'clone3', 'clone4', 'clone5', 'clone6', 'clone7', 'clone8', 'clone9', 'clone10']:
            
                try:
                    cn = intervals.iloc[0]['cn_{}'.format(idx)]
                    mu = intervals.iloc[0]['u_{}'.format(idx)]
                except: 
                    continue
                try:
                    cn_dict[cn] += mu
                except:
                    cn_dict[cn] = mu
            
        except IndexError:
            continue
            
        line = [sample_idx, sample, char_idx, char_label, ref, var]
        
        states6 = set()
        for cn in sorted(cn_dict):
            c1a = cn.split('|')[0]
            c1b = cn.split('|')[1]
            mu1 = cn_dict[cn]
            line += [c1a, c1b, mu1]
            highcn = highcn or (int(c1a) + int(c1b)) > 6
            if (int(c1a) + int(c1b)) == 6:
                states6.add((c1a, c1b))
        highcn = highcn or len(states6) > 1
        buff.append(line)
    
    if not highcn:
        input_data.extend(buff)
    else:
        discarded += 1
print 'Discarded {}'.format(discarded)

0 471
Discarded 870


In [6]:
## Generate the SNV input for DeCiFer with CNAs

with open(INPUT_SNVs, 'w') as out:
    out.write('{} #characters\n'.format(len(selected_ids)))
    out.write('{} #samples\n'.format(len(purities)))
    out.write("#sample_index	sample_label	character_index	character_label	ref	var\n")
    for line in input_data:
        out.write("\t".join(map(str, line))+"\n")       